In [1]:
import pyaudio
import wave
import sys
import os
import piplates.POWERplate as POW
import datetime
import logging
import datetime

# DOES NOT RUN ON UBUNTU 20.04 / PYTHON 3.9 BECAUSE I CAN'T GET pyaudiIo TO INSTALL PROPERLY 

On my Ubuntu 20.04 machine I tried to install pyaudio using
```
sudo apt-get install -y python3-pyaudio
```
FAILED

In [15]:
sys.version

'3.7.3 (default, Jan 22 2021, 20:04:44) \n[GCC 8.3.0]'

In [16]:
p = pyaudio.PyAudio()
for dev_index in range(p.get_device_count()):
    print(dev_index, p.get_device_info_by_index(dev_index).get('name'))

ALSA lib confmisc.c:1281:(snd_func_refer) Unable to find definition 'cards.bcm2835_hdmi.pcm.front.0:CARD=0'
ALSA lib conf.c:4568:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5047:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2565:(snd_pcm_open_noupdate) Unknown PCM front
ALSA lib pcm.c:2565:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.rear
ALSA lib pcm.c:2565:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.center_lfe
ALSA lib pcm.c:2565:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.side
ALSA lib confmisc.c:1281:(snd_func_refer) Unable to find definition 'cards.bcm2835_hdmi.pcm.surround51.0:CARD=0'
ALSA lib conf.c:4568:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5047:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2565:(snd_pcm_open_noupdate) Unknown PCM surround21
ALSA lib confmisc.c:1281:(snd_func_refer) Una

0 bcm2835 HDMI 1: - (hw:0,0)
1 Cable Creation: USB Audio (hw:1,0)
2 bcm2835 Headphones: - (hw:2,0)
3 sysdefault
4 lavrate
5 samplerate
6 speexrate
7 pulse
8 upmix
9 vdownmix
10 dmix
11 default


Cannot connect to server socket err = No such file or directory
Cannot connect to server request channel
jack server is not running or cannot be started
JackShmReadWritePtr::~JackShmReadWritePtr - Init not done for -1, skipping unlock
JackShmReadWritePtr::~JackShmReadWritePtr - Init not done for -1, skipping unlock


In [17]:
# These parameters may be set by papermill
AUDIO_FORMAT = pyaudio.paInt16    # 16-bit resolution
CHANNELS = 1                      # 1 channel
RATE = 44100                      # 44.1 kHz sampling rate
FRAMES_PER_BUFFER = 4096          # 2^12 samples for buffer
RECORD_SECS = 60                  # seconds to record
DEV_INDEX = 1                    # device index found by p.get_device_info_by_index()
# WAV_OUTPUT_FILENAME = 'lunga_up.wav' # name of .wav file

In [13]:
def record():
    audio = pyaudio.PyAudio() # create pyaudio instantiation

    # create pyaudio stream
    stream = audio.open(format = AUDIO_FORMAT,
                        rate = RATE,
                        channels = CHANNELS,
                        input_device_index = DEV_INDEX,
                        input = True,
                        frames_per_buffer = FRAMES_PER_BUFFER)

    frames = []
    filename = f'{timestamp()}.wav'
    logging.info(f'recording {filename}')
    
    # loop through stream and append audio chunks to frame array
    for chunk in range(0,int((RATE/FRAMES_PER_BUFFER)*RECORD_SECS)):
        data = stream.read(FRAMES_PER_BUFFER)
        frames.append(data)

    # stop the stream, close it, and terminate the pyaudio instantiation
    stream.stop_stream()
    stream.close()
    audio.terminate()

    # save the audio frames as .wav file
    wavefile = wave.open(filename,'wb')
    wavefile.setnchannels(CHANNELS)
    wavefile.setsampwidth(audio.get_sample_size(AUDIO_FORMAT))
    wavefile.setframerate(RATE)
    wavefile.writeframes(b''.join(frames))
    wavefile.close()

In [19]:
def update_github():
    os.system("git pull")
    os.system("git add .")
    os.system("git commit -m 'commited by record_wav.ipynb'")
    os.system("git push")
# update_github()

In [20]:
def metadata():
    return {'time':str(datetime.datetime.now()), 'volts':POW.getVin(0), 'amps':POW.getIin(0), 'temp':POW.getCPUtemp()}

# metadata()

In [21]:
def poweroff():
    POW.powerOFF(0)

# metadata()

In [11]:
def timestamp():
    '''returns local date and time'''
    return datetime.datetime.now().strftime("%Y%m%d_%H%M%S%f")

# timestamp()

'20221227_153123640082'

# MAIN

In [22]:
logging.basicConfig(filename = 'record_wav.log',
                    level = logging.DEBUG,
                    format = '%(asctime)s:%(levelname)s:%(name)s:%(message)s')
logging.info(metadata())
logging.info('recording')
record()
logging.info(metadata())
logging.info('updating github')
update_github()
logging.info(metadata())
logging.info('FINISHED')

recording
finished recording
Already up to date.
[master ee264e0] commited by record_wav.ipynb
 3 files changed, 104 insertions(+), 9 deletions(-)


To https://github.com/aubreymoore/IREC.git
   e789512..ee264e0  master -> master


In [23]:
print('FINISHED')

FINISHED
